In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import re
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import datetime as dt
from dateutil.parser import *
import random
import time
import json
from numpy import argmax
import warnings
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split,cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_auc_score,classification_report, roc_auc_score,accuracy_score, \
precision_score, f1_score, recall_score, roc_curve
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
import re
import seaborn as sns
from imblearn.over_sampling import SMOTE 

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier

%config InlineBackend.figure_format = 'retina'
plt.rc('xtick', labelsize=12) 
plt.rc('ytick', labelsize=12)

In [2]:
df_regression = pd.read_csv('../data/merged_train_final.csv')

In [3]:
test_df_regression = pd.read_csv('../data/merged_test_final.csv')

In [4]:
merged_train_multi = pd.read_csv('../data/merged_train_multi.csv')

In [5]:
merged_test_multi = pd.read_csv('../data/merged_test_multi.csv')

In [6]:
x = df_regression.drop(['wnvpresent'],axis=1)
y = df_regression['wnvpresent']

#x = merged_train_multi.drop(['wnvpresent'],axis=1)
#y = merged_train_multi['wnvpresent']

display(y.value_counts(normalize=True))

x_train, x_test, y_train, y_test= train_test_split(x,y,random_state=42)

ss = StandardScaler()
x_train_sc = ss.fit_transform(x_train)
x_test_sc = ss.transform(x_test)

sm = SMOTE(random_state=42)

x_train_sc, y_train = sm.fit_resample(x_train_sc,y_train)

0    0.948107
1    0.051893
Name: wnvpresent, dtype: float64

In [7]:
# Instantiate models
models = {'lr': LogisticRegression(max_iter=5_000, random_state=42, solver='saga'),
          'rf': RandomForestClassifier(random_state=42),
          'gb': GradientBoostingClassifier(random_state=42),
          'et': ExtraTreesClassifier(random_state=42),
          'ada': AdaBoostClassifier(random_state=42),
          'svm': SVC(random_state=42, probability=True),
          'xg': XGBClassifier(random_state=42)
}

In [8]:
# Function to run model -- input model
def get_model_scores(model_name,
                     mod, 
                     mod_params={}, 
                     grid_search=False):
    
    """Function accepts following inputs:
    Name of model (str), model to be used (str), 
    model params(dict, optional), grid_seach(boolean, optional)
    If grid_search is True, then please also input the relevant 
    params for GridSearching
    """
    
    # empty dict for appending results
    results = {}
    
    # instantiate pipe
    pipe = Pipeline([
            (mod, models[mod])
            ])
    
    # check if GridSearch true or false
    if grid_search:
        
        # combine vectorizer and mod params together
        gs_params = {}
        gs_params.update(mod_params)
        
        # instantiate GridSearchCV
        gs = GridSearchCV(pipe, 
                          param_grid=gs_params,
                          cv=5, 
                          verbose=2, 
                          n_jobs=-1)
        
        # fit model
        gs.fit(x_train_sc, y_train)
        pipe = gs
        
    else:
        # else fit model
        pipe.fit(x_train_sc, y_train)
    
    # create predictions and confusion matrix
    predictions = pipe.predict(x_test_sc)
    tn, fp, fn, tp = confusion_matrix(y_test, 
                                      predictions).ravel()
    y_test_pred_prob = pipe.predict_proba(x_test_sc)[:,1]
    y_train_pred_prob = pipe.predict_proba(x_train_sc)[:,1]

    train_auc = roc_auc_score(y_train, y_train_pred_prob)
    test_auc = roc_auc_score(y_test, y_test_pred_prob)
    
    # Retrieve metrics and add to results
    results['model_name'] = model_name
    results['model'] = mod
    results['train_score'] = pipe.score(x_train_sc, y_train)
    results['test_score'] = pipe.score(x_test_sc, y_test)
    
    results['recall'] = recall_score(y_test, 
                                     predictions)
    
    results['specificity'] = tn/(tn + fp)
    
    results['precision'] = precision_score(y_test, 
                                           predictions)
    
    results['train_auc'] = roc_auc_score(y_train, y_train_pred_prob)
    results['test_auc'] = roc_auc_score(y_test, y_test_pred_prob)
    
    results['is_tuned'] = grid_search
    
    if grid_search:
        print('BEST PARAMS-->')
        display(pipe.best_params_)
    
    # add results to list for model evaluation later
    model_eval.append(results)
    
    print('METRICS-->')
    display(results)
    
    tn, fp, fn, tp = confusion_matrix(y_test, 
                                      predictions).ravel()
    
    print(f"True Negatives: {tn}")
    print(f"False Positives: {fp}")
    print(f"False Negatives: {fn}")
    print(f"True Positives: {tp}")
    
    return pipe

In [9]:
# create list to store results
model_eval =[]

In [10]:
lr_reg = get_model_scores('lr_base', 'lr')

METRICS-->


{'model_name': 'lr_base',
 'model': 'lr',
 'train_score': 0.8057329462989841,
 'test_score': 0.7541254125412541,
 'recall': 0.5887096774193549,
 'specificity': 0.7630434782608696,
 'precision': 0.11812297734627832,
 'train_auc': 0.8751527212826059,
 'test_auc': 0.7461448106591865,
 'is_tuned': False}

True Negatives: 1755
False Positives: 545
False Negatives: 51
True Positives: 73


In [11]:
svm_reg = get_model_scores('svm_base', 'svm')

METRICS-->


{'model_name': 'svm_base',
 'model': 'svm',
 'train_score': 0.9130624092888244,
 'test_score': 0.8407590759075908,
 'recall': 0.4274193548387097,
 'specificity': 0.8630434782608696,
 'precision': 0.14402173913043478,
 'train_auc': 0.9712810577160058,
 'test_auc': 0.7514708976157083,
 'is_tuned': False}

True Negatives: 1985
False Positives: 315
False Negatives: 71
True Positives: 53


In [12]:
xg_reg = get_model_scores('xg_base', 'xg')

[20:43:11] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/user_1/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


METRICS-->


{'model_name': 'xg_base',
 'model': 'xg',
 'train_score': 0.9776487663280116,
 'test_score': 0.9426567656765676,
 'recall': 0.08064516129032258,
 'specificity': 0.9891304347826086,
 'precision': 0.2857142857142857,
 'train_auc': 0.9971802490304831,
 'test_auc': 0.8572072230014025,
 'is_tuned': False}

True Negatives: 2275
False Positives: 25
False Negatives: 114
True Positives: 10


In [13]:
et_reg = get_model_scores('et_base', 'et')

METRICS-->


{'model_name': 'et_base',
 'model': 'et',
 'train_score': 0.9918722786647315,
 'test_score': 0.941006600660066,
 'recall': 0.07258064516129033,
 'specificity': 0.9878260869565217,
 'precision': 0.24324324324324326,
 'train_auc': 0.9998393056131918,
 'test_auc': 0.6866374474053296,
 'is_tuned': False}

True Negatives: 2272
False Positives: 28
False Negatives: 115
True Positives: 9


In [14]:
model_eval

[{'model_name': 'lr_base',
  'model': 'lr',
  'train_score': 0.8057329462989841,
  'test_score': 0.7541254125412541,
  'recall': 0.5887096774193549,
  'specificity': 0.7630434782608696,
  'precision': 0.11812297734627832,
  'train_auc': 0.8751527212826059,
  'test_auc': 0.7461448106591865,
  'is_tuned': False},
 {'model_name': 'svm_base',
  'model': 'svm',
  'train_score': 0.9130624092888244,
  'test_score': 0.8407590759075908,
  'recall': 0.4274193548387097,
  'specificity': 0.8630434782608696,
  'precision': 0.14402173913043478,
  'train_auc': 0.9712810577160058,
  'test_auc': 0.7514708976157083,
  'is_tuned': False},
 {'model_name': 'xg_base',
  'model': 'xg',
  'train_score': 0.9776487663280116,
  'test_score': 0.9426567656765676,
  'recall': 0.08064516129032258,
  'specificity': 0.9891304347826086,
  'precision': 0.2857142857142857,
  'train_auc': 0.9971802490304831,
  'test_auc': 0.8572072230014025,
  'is_tuned': False},
 {'model_name': 'et_base',
  'model': 'et',
  'train_score'

In [103]:
#x = df_regression.drop(['wnvpresent'],axis=1)
#y = df_regression['wnvpresent']

x = merged_train_multi.drop(['wnvpresent'],axis=1)
y = merged_train_multi['wnvpresent']

display(y.value_counts(normalize=True))

x_train, x_test, y_train, y_test= train_test_split(x,y,random_state=42)

ss = StandardScaler()
x_train_sc = ss.fit_transform(x_train)
x_test_sc = ss.transform(x_test)

sm = SMOTE(random_state=42)

x_train_sc, y_train = sm.fit_resample(x_train_sc,y_train)

0    0.948107
1    0.051893
Name: wnvpresent, dtype: float64

In [104]:
# Function to run model -- input model
def get_model_scores(model_name,
                     mod, 
                     mod_params={}, 
                     grid_search=False):
    
    """Function accepts following inputs:
    Name of model (str), model to be used (str), 
    model params(dict, optional), grid_seach(boolean, optional)
    If grid_search is True, then please also input the relevant 
    params for GridSearching
    """
    
    # empty dict for appending results
    results = {}
    
    # instantiate pipe
    pipe = Pipeline([
            (mod, models[mod])
            ])
    
    # check if GridSearch true or false
    if grid_search:
        
        # combine vectorizer and mod params together
        gs_params = {}
        gs_params.update(mod_params)
        
        # instantiate GridSearchCV
        gs = GridSearchCV(pipe, 
                          param_grid=gs_params,
                          cv=5, 
                          verbose=2, 
                          n_jobs=-1,
                          scoring ='recall'
                          
                         )
        
        # fit model
        gs.fit(x_train_sc, y_train)
        pipe = gs
        
    else:
        # else fit model
        pipe.fit(x_train_sc, y_train)
    
    # create predictions and confusion matrix
    predictions = pipe.predict(x_test_sc)
    tn, fp, fn, tp = confusion_matrix(y_test, 
                                      predictions).ravel()
    y_test_pred_prob = pipe.predict_proba(x_test_sc)[:,1]
    y_train_pred_prob = pipe.predict_proba(x_train_sc)[:,1]

    train_auc = roc_auc_score(y_train, y_train_pred_prob)
    test_auc = roc_auc_score(y_test, y_test_pred_prob)
    
    # Retrieve metrics and add to results
    results['model_name'] = model_name
    results['model'] = mod
    results['train_score'] = pipe.score(x_train_sc, y_train)
    results['test_score'] = pipe.score(x_test_sc, y_test)
    
    results['recall'] = recall_score(y_test, 
                                     predictions)
    
    results['specificity'] = tn/(tn + fp)
    
    results['precision'] = precision_score(y_test, 
                                           predictions)
    
    results['train_auc'] = roc_auc_score(y_train, y_train_pred_prob)
    results['test_auc'] = roc_auc_score(y_test, y_test_pred_prob)
    
    results['is_tuned'] = grid_search
    
    if grid_search:
        print('BEST PARAMS-->')
        display(pipe.best_params_)
    
    # add results to list for model evaluation later
    model_eval_2.append(results)
    
    print('METRICS-->')
    display(results)
    
    tn, fp, fn, tp = confusion_matrix(y_test, 
                                      predictions).ravel()
    
    print(f"True Negatives: {tn}")
    print(f"False Positives: {fp}")
    print(f"False Negatives: {fn}")
    print(f"True Positives: {tp}")
    
    return pipe

In [105]:
model_eval_2 = []

In [106]:
lr_base = get_model_scores('lr_base', 'lr')

METRICS-->


{'model_name': 'lr_base',
 'model': 'lr',
 'train_score': 0.8259796806966618,
 'test_score': 0.775990099009901,
 'recall': 0.6048387096774194,
 'specificity': 0.7852173913043479,
 'precision': 0.13181019332161686,
 'train_auc': 0.8874044228083443,
 'test_auc': 0.7729926367461429,
 'is_tuned': False}

True Negatives: 1806
False Positives: 494
False Negatives: 49
True Positives: 75


In [107]:
svm_base = get_model_scores('svm_base', 'svm')

METRICS-->


{'model_name': 'svm_base',
 'model': 'svm',
 'train_score': 0.9298984034833091,
 'test_score': 0.8535478547854786,
 'recall': 0.45161290322580644,
 'specificity': 0.8752173913043478,
 'precision': 0.16326530612244897,
 'train_auc': 0.9783968162352202,
 'test_auc': 0.777312412342216,
 'is_tuned': False}

True Negatives: 2013
False Positives: 287
False Negatives: 68
True Positives: 56


In [108]:
xg_base = get_model_scores('xg_base', 'xg')

[00:00:46] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/user_1/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


METRICS-->


{'model_name': 'xg_base',
 'model': 'xg',
 'train_score': 0.978011611030479,
 'test_score': 0.9422442244224423,
 'recall': 0.0967741935483871,
 'specificity': 0.9878260869565217,
 'precision': 0.3,
 'train_auc': 0.9974362836276467,
 'test_auc': 0.8528962131837308,
 'is_tuned': False}

True Negatives: 2272
False Positives: 28
False Negatives: 112
True Positives: 12


In [109]:
model_eval_2

[{'model_name': 'lr_base',
  'model': 'lr',
  'train_score': 0.8259796806966618,
  'test_score': 0.775990099009901,
  'recall': 0.6048387096774194,
  'specificity': 0.7852173913043479,
  'precision': 0.13181019332161686,
  'train_auc': 0.8874044228083443,
  'test_auc': 0.7729926367461429,
  'is_tuned': False},
 {'model_name': 'svm_base',
  'model': 'svm',
  'train_score': 0.9298984034833091,
  'test_score': 0.8535478547854786,
  'recall': 0.45161290322580644,
  'specificity': 0.8752173913043478,
  'precision': 0.16326530612244897,
  'train_auc': 0.9783968162352202,
  'test_auc': 0.777312412342216,
  'is_tuned': False},
 {'model_name': 'xg_base',
  'model': 'xg',
  'train_score': 0.978011611030479,
  'test_score': 0.9422442244224423,
  'recall': 0.0967741935483871,
  'specificity': 0.9878260869565217,
  'precision': 0.3,
  'train_auc': 0.9974362836276467,
  'test_auc': 0.8528962131837308,
  'is_tuned': False}]

In [110]:
# Function to run model -- input model
def get_model_scores_roc(model_name,
                     mod, 
                     mod_params={}, 
                     grid_search=False):
    
    """Function accepts following inputs:
    Name of model (str), model to be used (str), 
    model params(dict, optional), grid_seach(boolean, optional)
    If grid_search is True, then please also input the relevant 
    params for GridSearching
    """
    
    # empty dict for appending results
    results = {}
    
    # instantiate pipe
    pipe = Pipeline([
            (mod, models[mod])
            ])
    
    # check if GridSearch true or false
    if grid_search:
        
        # combine vectorizer and mod params together
        gs_params = {}
        gs_params.update(mod_params)
        
        # instantiate GridSearchCV
        gs = GridSearchCV(pipe, 
                          param_grid=gs_params,
                          cv=5, 
                          verbose=2, 
                          n_jobs=-1,
                          scoring ='roc_auc'
                          
                         )
        
        # fit model
        gs.fit(x_train_sc, y_train)
        pipe = gs
        
    else:
        # else fit model
        pipe.fit(x_train_sc, y_train)
    
    # create predictions and confusion matrix
    predictions = pipe.predict(x_test_sc)
    tn, fp, fn, tp = confusion_matrix(y_test, 
                                      predictions).ravel()
    y_test_pred_prob = pipe.predict_proba(x_test_sc)[:,1]
    y_train_pred_prob = pipe.predict_proba(x_train_sc)[:,1]

    train_auc = roc_auc_score(y_train, y_train_pred_prob)
    test_auc = roc_auc_score(y_test, y_test_pred_prob)
    
    # Retrieve metrics and add to results
    results['model_name'] = model_name
    results['model'] = mod
    results['train_score'] = pipe.score(x_train_sc, y_train)
    results['test_score'] = pipe.score(x_test_sc, y_test)
    
    results['recall'] = recall_score(y_test, 
                                     predictions)
    
    results['specificity'] = tn/(tn + fp)
    
    results['precision'] = precision_score(y_test, 
                                           predictions)
    
    results['train_auc'] = roc_auc_score(y_train, y_train_pred_prob)
    results['test_auc'] = roc_auc_score(y_test, y_test_pred_prob)
    
    results['is_tuned'] = grid_search
    
    if grid_search:
        print('BEST PARAMS-->')
        display(pipe.best_params_)
    
    # add results to list for model evaluation later
    model_eval.append(results)
    
    print('METRICS-->')
    display(results)
    
    tn, fp, fn, tp = confusion_matrix(y_test, 
                                      predictions).ravel()
    
    print(f"True Negatives: {tn}")
    print(f"False Positives: {fp}")
    print(f"False Negatives: {fn}")
    print(f"True Positives: {tp}")
    
    return pipe

In [111]:
# Function to run model -- input model
def get_model_scores_recall(model_name,
                     mod, 
                     mod_params={}, 
                     grid_search=False):
    
    """Function accepts following inputs:
    Name of model (str), model to be used (str), 
    model params(dict, optional), grid_seach(boolean, optional)
    If grid_search is True, then please also input the relevant 
    params for GridSearching
    """
    
    # empty dict for appending results
    results = {}
    
    # instantiate pipe
    pipe = Pipeline([
            (mod, models[mod])
            ])
    
    # check if GridSearch true or false
    if grid_search:
        
        # combine vectorizer and mod params together
        gs_params = {}
        gs_params.update(mod_params)
        
        # instantiate GridSearchCV
        gs = GridSearchCV(pipe, 
                          param_grid=gs_params,
                          cv=5, 
                          verbose=2, 
                          n_jobs=-1,
                          scoring ='recall'
                          
                         )
        
        # fit model
        gs.fit(x_train_sc, y_train)
        pipe = gs
        
    else:
        # else fit model
        pipe.fit(x_train_sc, y_train)
    
    # create predictions and confusion matrix
    predictions = pipe.predict(x_test_sc)
    tn, fp, fn, tp = confusion_matrix(y_test, 
                                      predictions).ravel()
    y_test_pred_prob = pipe.predict_proba(x_test_sc)[:,1]
    y_train_pred_prob = pipe.predict_proba(x_train_sc)[:,1]

    train_auc = roc_auc_score(y_train, y_train_pred_prob)
    test_auc = roc_auc_score(y_test, y_test_pred_prob)
    
    # Retrieve metrics and add to results
    results['model_name'] = model_name
    results['model'] = mod
    results['train_score'] = pipe.score(x_train_sc, y_train)
    results['test_score'] = pipe.score(x_test_sc, y_test)
    
    results['recall'] = recall_score(y_test, 
                                     predictions)
    
    results['specificity'] = tn/(tn + fp)
    
    results['precision'] = precision_score(y_test, 
                                           predictions)
    
    results['train_auc'] = roc_auc_score(y_train, y_train_pred_prob)
    results['test_auc'] = roc_auc_score(y_test, y_test_pred_prob)
    
    results['is_tuned'] = grid_search
    
    if grid_search:
        print('BEST PARAMS-->')
        display(pipe.best_params_)
    
    # add results to list for model evaluation later
    model_eval.append(results)
    
    print('METRICS-->')
    display(results)
    
    tn, fp, fn, tp = confusion_matrix(y_test, 
                                      predictions).ravel()
    
    print(f"True Negatives: {tn}")
    print(f"False Positives: {fp}")
    print(f"False Negatives: {fn}")
    print(f"True Positives: {tp}")
    
    return pipe

In [112]:
model_eval =[]

In [113]:
lr_params = {
    # Trying different types of regularization
    'lr__penalty':['l2','l1', 'elasticnet'],

     # Trying different alphas of: 1, 0.1, 0.05  (C = 1/alpha)
    'lr__C':[1, 10, 20],

}


svc_params = {
    'svm__C':[1, 10],
    'svm__gamma':['scale', 'auto'], 
    'svm__kernel':['rbf', 'sigmoid'],
}

xg_params = {
    'xg__booster': ['gbtree', 'gblinear'],
    'xg__scale_pos_weight': [0.5,1],
    'xg__gamma': [0,1],
    'xg__reg_alpha': [0, 1],
    'xg__reg_lambda': [1, 5]}

In [114]:
lr_tune = get_model_scores_roc('lr_tune', 'lr',lr_params,True)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


/Users/user_1/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
15 fits failed out of a total of 45.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/user_1/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/user_1/opt/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/user_1/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1471, in fit
  

BEST PARAMS-->


/Users/user_1/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


{'lr__C': 20, 'lr__penalty': 'l1'}

METRICS-->


{'model_name': 'lr_tune',
 'model': 'lr',
 'train_score': 0.8894924071191288,
 'test_score': 0.7765971248246846,
 'recall': 0.5887096774193549,
 'specificity': 0.7882608695652173,
 'precision': 0.13035714285714287,
 'train_auc': 0.8894924071191288,
 'test_auc': 0.7765971248246846,
 'is_tuned': True}

True Negatives: 1813
False Positives: 487
False Negatives: 51
True Positives: 73


In [115]:
lr_tune_recall = get_model_scores_recall('lr_tune_recall', 'lr',lr_params,True)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


/Users/user_1/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
15 fits failed out of a total of 45.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/user_1/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/user_1/opt/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/user_1/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1471, in fit
  

BEST PARAMS-->


/Users/user_1/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


{'lr__C': 10, 'lr__penalty': 'l2'}

METRICS-->


{'model_name': 'lr_tune_recall',
 'model': 'lr',
 'train_score': 0.8795355587808418,
 'test_score': 0.5887096774193549,
 'recall': 0.5887096774193549,
 'specificity': 0.7882608695652173,
 'precision': 0.13035714285714287,
 'train_auc': 0.8893119010955909,
 'test_auc': 0.7761553295932678,
 'is_tuned': True}

True Negatives: 1813
False Positives: 487
False Negatives: 51
True Positives: 73


In [116]:
svm_tune = get_model_scores_roc('svm_tune', 'svm',svc_params,True)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


/Users/user_1/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


BEST PARAMS-->


{'svm__C': 10, 'svm__gamma': 'scale', 'svm__kernel': 'rbf'}

METRICS-->


{'model_name': 'svm_tune',
 'model': 'svm',
 'train_score': 0.9935818196372185,
 'test_score': 0.7764638849929875,
 'recall': 0.3548387096774194,
 'specificity': 0.9265217391304348,
 'precision': 0.20657276995305165,
 'train_auc': 0.9935817353772005,
 'test_auc': 0.7742917251051893,
 'is_tuned': True}

True Negatives: 2131
False Positives: 169
False Negatives: 80
True Positives: 44


In [117]:
svm_tune_recall = get_model_scores_recall('svm_tune_recall', 'svm',svc_params,True)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
BEST PARAMS-->


{'svm__C': 10, 'svm__gamma': 'scale', 'svm__kernel': 'rbf'}

METRICS-->


{'model_name': 'svm_tune_recall',
 'model': 'svm',
 'train_score': 0.997532656023222,
 'test_score': 0.3548387096774194,
 'recall': 0.3548387096774194,
 'specificity': 0.9265217391304348,
 'precision': 0.20657276995305165,
 'train_auc': 0.9935817353772005,
 'test_auc': 0.7742917251051893,
 'is_tuned': True}

True Negatives: 2131
False Positives: 169
False Negatives: 80
True Positives: 44


In [118]:
xgb_tune = get_model_scores_roc('xg_tune', 'xg',xg_params,True)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


/Users/user_1/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:40:47] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
BEST PARAMS-->


{'xg__booster': 'gbtree',
 'xg__gamma': 1,
 'xg__reg_alpha': 0,
 'xg__reg_lambda': 1,
 'xg__scale_pos_weight': 0.5}

METRICS-->


{'model_name': 'xg_tune',
 'model': 'xg',
 'train_score': 0.995142209845362,
 'test_score': 0.8477857643758766,
 'recall': 0.056451612903225805,
 'specificity': 0.9947826086956522,
 'precision': 0.3684210526315789,
 'train_auc': 0.995142209845362,
 'test_auc': 0.8477857643758766,
 'is_tuned': True}

True Negatives: 2288
False Positives: 12
False Negatives: 117
True Positives: 7


In [121]:
xgb_tune_recall = get_model_scores_recall('xg_tune_recall', 'xg',xg_params,True)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


/Users/user_1/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:37:37] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
BEST PARAMS-->


{'xg__booster': 'gbtree',
 'xg__gamma': 0,
 'xg__reg_alpha': 0,
 'xg__reg_lambda': 1,
 'xg__scale_pos_weight': 1}

METRICS-->


{'model_name': 'xg_tune_recall',
 'model': 'xg',
 'train_score': 0.9631349782293178,
 'test_score': 0.0967741935483871,
 'recall': 0.0967741935483871,
 'specificity': 0.9878260869565217,
 'precision': 0.3,
 'train_auc': 0.9974362836276467,
 'test_auc': 0.8528962131837308,
 'is_tuned': True}

True Negatives: 2272
False Positives: 28
False Negatives: 112
True Positives: 12


In [127]:
pd.DataFrame(model_eval).sort_values('recall',ascending=False)

,model_name,model,train_score,test_score,recall,specificity,precision,train_auc,test_auc,is_tuned
0,lr_tune,lr,0.889492,0.776597,0.588710,0.788261,0.130357,0.889492,0.776597,True
1,lr_tune_recall,lr,0.879536,0.588710,0.588710,0.788261,0.130357,0.889312,0.776155,True
2,svm_tune,svm,0.993582,0.776464,0.354839,0.926522,0.206573,0.993582,0.774292,True
3,svm_tune_recall,svm,0.997533,0.354839,0.354839,0.926522,0.206573,0.993582,0.774292,True
5,xg_tune_recall,xg,0.963135,0.096774,0.096774,0.987826,0.300000,0.997436,0.852896,True
4,xg_tune,xg,0.995142,0.847786,0.056452,0.994783,0.368421,0.995142,0.847786,True


In [128]:
pd.DataFrame(model_eval).sort_values('test_auc',ascending=False)

,model_name,model,train_score,test_score,recall,specificity,precision,train_auc,test_auc,is_tuned
5,xg_tune_recall,xg,0.963135,0.096774,0.096774,0.987826,0.300000,0.997436,0.852896,True
4,xg_tune,xg,0.995142,0.847786,0.056452,0.994783,0.368421,0.995142,0.847786,True
0,lr_tune,lr,0.889492,0.776597,0.588710,0.788261,0.130357,0.889492,0.776597,True
1,lr_tune_recall,lr,0.879536,0.588710,0.588710,0.788261,0.130357,0.889312,0.776155,True
2,svm_tune,svm,0.993582,0.776464,0.354839,0.926522,0.206573,0.993582,0.774292,True
3,svm_tune_recall,svm,0.997533,0.354839,0.354839,0.926522,0.206573,0.993582,0.774292,True
